In [1]:
!pip install langchain huggingface_hub sentence_transformers faiss-gpu langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import langchain
from langchain_groq import ChatGroq
import getpass
import os
from langchain import PromptTemplate, HuggingFaceHub
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain.vectorstores import FAISS
import numpy as np

In [5]:
inference_api_key = getpass.getpass("Enter your HF Inference API Key:\n\n")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = inference_api_key

groq_api_key = getpass.getpass("Enter your Groq API Key:\n\n")

Enter your HF Inference API Key:

··········
Enter your Groq API Key:

··········


In [6]:
chat = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama2-70b-4096", max_tokens = 1000)
#print(chat.invoke("Who is Usain Bolt"))

In [7]:
embeddings_model_name = "mixedbread-ai/mxbai-embed-large-v1"

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/112k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [11]:
index_loc = "/content/drive/MyDrive/faiss_percy_jackson_index"
index = FAISS.load_local(index_loc, embeddings,allow_dangerous_deserialization=True)


In [17]:
retriever = index.as_retriever()
retriever.search_kwargs['fetch_k'] = 30
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 15

In [18]:
story_template = '''You are a creative and helpful storyteller AI. When given context and question, you get a creative and in depth story based on the question.

context = {context}

question = Write a detailed story based on the context for {question}

Answer:
'''

story_prompt = PromptTemplate(input_variables = ['context','question'], template = story_template)

In [19]:
chain_story = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": story_prompt},
    verbose=True
)

handler = StdOutCallbackHandler()

print(chain_story.invoke(
    'Percy nd Grover hanging the casino?',
    callbacks=[handler]
))



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'Percy nd Grover hanging the casino?', 'result': 'Percy and Grover walked into the casino, looking around in awe at the flashing lights and ringing machines. They had never seen anything like it before. Grover, being a satyr, had never been to a place like this before, and Percy, being a demigod, had never been allowed to enter a place like this. But they were on a mission. They had heard that the casino was run by a powerful demon who was using it to lure in unsuspecting humans and monsters alike, and they were determined to put a stop to it.\n\nAs they made their way through the crowds of people, Percy noticed a group of monsters huddled in the corner, muttering to each other. He recognized them as some of the same monsters that had attacked him and his friends in the past. He knew they were up to no good, and he motioned to Grover to follow him.\n\nThey approached the group, and Percy demanded to know what they were 

In [20]:
question_template = '''You are a smart AI who responds based on the provided context. When given context and question, you get a creative and in depth explanation based on the question and context.

context = {context}

question = Write a detailed explanation based on the context for {question}

Answer:
'''

question_prompt = PromptTemplate(input_variables = ['context','question'], template = question_template)

In [21]:
chain_query = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": question_prompt},
    verbose=True
)

handler = StdOutCallbackHandler()

print(chain_query.invoke(
    'Who is Tyson?',
    callbacks=[handler]
))



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'Who is Tyson?', 'result': 'Tyson is a unique young man who is a student at Meriwether College Prep. He is described as being very tall, standing at six-foot-three, and having a large, calf-brown eye in the middle of his forehead. Tyson is a Cyclops, a creature from Greek mythology known for having a single eye in the middle of their forehead. Despite his intimidating appearance, Tyson is a kind and gentle soul who is often bullied by his classmates. He is also incredibly strong and has the ability to mimic voices, which he uses to communicate with others. Tyson is a friend of the protagonist, Percy Jackson, and is fiercely loyal to him. He is also a demigod, meaning he is the child of a mortal and a god, and is destined to play an important role in the world of Greek mythology.'}


In [22]:
print(chain_query.invoke(
    'Who is Nico de Angelo?',
    callbacks=[handler]
))



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'Who is Nico de Angelo?', 'result': 'Nico di Angelo is a young boy who is the son of Hades and a mortal woman named Maria di Angelo. He has dark hair, pale skin, and dark clothes, and he wields a short, sharp sword that is said to be able to absorb the essence of monsters. Nico is a powerful demigod who has the ability to control the dead and can summon the spirits of the Underworld. He is on a quest to bring his sister, Bianca, back from the dead, and he is willing to do whatever it takes to achieve his goal, even if it means sacrificing his own life. Nico is also a skilled Mythomagic player, and he uses his knowledge of the game to navigate the Underworld and communicate with the spirits of the dead. Despite his young age, Nico is fiercely independent and has a strong sense of determination and purpose. He is also shown to be protective of his friends and will go to great lengths to keep them safe.'}
